In [1]:
# 📌 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install faiss-cpu
!pip install sentence-transformers
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
import pandas as pd

# Load filtered complaints
file_path = '/content/drive/My Drive/filtered_complaints.csv'
df = pd.read_csv(file_path)
df = df[['cleaned_narrative', 'Product']]  # Include any other metadata like complaint ID if needed

df.dropna(subset=['cleaned_narrative'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()


,cleaned_narrative,Product
0,a xxxx xxxx card was opened under my name by a...,Credit card
1,dear cfpb i have a secured credit card with ci...,Credit card
2,i have a citi rewards cards the credit balance...,Credit card
3,bi am writing to dispute the following charges...,Credit card
4,although the account had been deemed closed i ...,Credit card


In [4]:
def chunk_text(text, chunk_size=300, chunk_overlap=50):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i + chunk_size]
        chunks.append(' '.join(chunk))
        i += chunk_size - chunk_overlap
    return chunks


In [5]:
chunk_size = 300
chunk_overlap = 50

chunked_texts = []
metadatas = []

for idx, row in df.iterrows():
    chunks = chunk_text(row['cleaned_narrative'], chunk_size, chunk_overlap)
    for chunk in chunks:
        chunked_texts.append(chunk)
        metadatas.append({
            'product': row['Product'],
            'original_index': idx
        })

print(f"Total chunks created: {len(chunked_texts)}")


Total chunks created: 109713


In [6]:
from sentence_transformers import SentenceTransformer

# Use all-MiniLM-L6-v2 for speed and performance balance
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(chunked_texts, show_progress_bar=True, convert_to_numpy=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3429 [00:00<?, ?it/s]

In [8]:
import faiss
import os
import pickle

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Save index
os.makedirs('vector_store', exist_ok=True)
faiss.write_index(index, 'vector_store/faiss_index.index')

# Save metadata separately
with open('vector_store/metadata.pkl', 'wb') as f:
    pickle.dump(metadatas, f)

print("Vector store and metadata saved in 'vector_store/' directory.")


Vector store and metadata saved in 'vector_store/' directory.
